### Eric He
### eh1885
### Deep Learning
### Homework 1: Backpropagation

# 1. Two-layer neural network

## 1.1 Regression task

### 1.1.a Name the 5 programming steps you would take to train this model with PyTorch using SGD on a single batch of data
Suppose we are provided `model`, a Torch model object, a loss function given by `l(y_pred, y_true)`, and a learning rate given by `lr`.

```
import torch
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=lr)

y_pred = model.forward() # 1. forward pass 
loss = l(y_pred, y_true) # 2. compute loss
optimizer.zero_grad() # 3. zero the grad-parameters
loss.backward() # 4. accumulate the grad-parameters
optimizer.step() # 5. step in the opposite direction of the grad-parameters
```

### 1.1.b For a single data point `(x, y)`, write down all inputs and outputs for forward pass of each layer. You can only use `x, y, W^1, b^1, W^2, b^2` in your answer.

#### Linear 1
Input: $x$

Output: $W^1x + b^1$

#### f
Input: $W^1x + b^1$

Output: $f(W^1x + b^1)$

#### Linear 2
Input: $f(W^1x + b^1)$

Output: $W^2(f(W^1x + b^1)) + b^2$

#### g
Input: $W^2(f(W^1x + b^1)) + b^2$

Output: $g(W^2(f(W^1x + b^1)) + b^2)$

#### Loss
Input: $g(W^2(f(W^1x + b^1)) + b^2)$

Output: $\dfrac{1}{2}(g(W^2(f(W^1x + b^1)) + b^2) - y)^2$

### 1.1.c Write down the gradient calculated from the backward pass. You can only use $x, y, W^1, b^1, W^2, b^2, \dfrac{\delta l}{\delta y}, \dfrac{\delta z_2}{\delta z_1}, \dfrac{\delta \hat{y}}{\delta z_3}$ in your answer, where $z_1, z_2, z_3, \hat{y}$ are the outputs of `Linear_1, f, Linear_2, g`.

For convenience, assume $W^1$ is a matrix of shape $(m_1, n_1)$ and assume $W^2$ is a matrix of shape $(m_2, n_2)$. Denote $I_{m_1}$ to be the identity matrix of shape $(m_1, m_1)$ and $I_{m_2}$ to be the identity matrix of shape $(m_2, m_2)$.

#### W1
We first expand the derivative:
$$
\dfrac{\delta l}{\delta W^1} = \dfrac{\delta l}{\delta \hat{y}}
    \dfrac{\delta \hat{y}}{\delta z_3}
    \dfrac{\delta z_3}{\delta z_2} 
    \dfrac{\delta z_2}{\delta z_1}
    \dfrac{\delta z_1}{\delta W^1}
 $$

We now have to fill in the gradients $\dfrac{\delta z_3}{\delta z_2}$ and $\dfrac{\delta z_1}{\delta W^1}$.

As $z_3 = W^2 z_2 + b^2$, we have $\dfrac{\delta z_3}{\delta z_2} = W^2$. 

As $z_1 = W^1 x + b^1$, we have $\dfrac{\delta z_1}{\delta W^1} = I_{m_1}x^T$. Notice that I have written this gradient as a matrix where $x^T$ is repeated several $m_1$ times; strictly speaking, because $z_1$ is a vector of shape $m_1$ and $W_1$ is a matrix of shape $(m_1, m_2)$, the Jacobian is of shape $(m_1, m_1, m_2)$. However, because $W_1$ is linear, the Jacobian is diagonal along one of the $m_1$ dimensions and I was able to squash it to a matrix with no repercussions.

Then the entire computation is 

$$
\dfrac{\delta l}{\delta W^1} = \dfrac{\delta l}{\delta \hat{y}}
    \dfrac{\delta \hat{y}}{\delta z_3}
    W^2
    \dfrac{\delta z_2}{\delta z_1}
    I_{m_1}x^T
 $$

### 1.1.d Show us the elements of $\dfrac{\delta z_2}{\delta z_1}, \dfrac{\delta \hat{y}}{\delta z_3}$ and $\dfrac{\delta l}{\delta \hat{y}}$?

$\dfrac{\delta z_1}{\delta z_1}$ is the derivative of the ReLU operation applied elementwise to the vector $z_1$. The subgradient of ReLU(x) is $1$ if $x > 0$ and $0$ otherwise, so we can write $\dfrac{\delta z_2}{\delta z_1} = 1_{z_{1} > 0}$, the indicator function elementwise for if $z_{1i} > 0$.

$\dfrac{\delta \hat{y}}{\delta z_3}$ is the derivative of the identity function of $z_3$. The identity function has derivative $1$, the identity matrix.

$\dfrac{\delta l}{\delta \hat{y}}$ is the derivative of the square loss, and is given by $\hat{y} - y$.

## 1.2 Classification Task

### 1.2.a If you want to train this network, what do you need to change in the equations of (b), (c), and (d), assuming we are using the same MLE loss function.
The forward pass would look the same, of course as I retained the $f$ and $g$ notations in the original answer. However, instead of $f(W^1x + b^1) = \max(0, W^1x + b^1)$, we now have  $f(W^1x + b^1) = \sigma(W^1x + b^1)$. And instead of $g(W^2(f(W^1x + b^1)) + b^2) = W^2(f(W^1x + b^1)) + b^2$, $g_2$ is now $\sigma(W^2(f(W^1x + b^1)) + b^2)$

The backward pass now involves the deriving the logistic sigmoid function, $\sigma(z) = (1 + \exp(-z))^{-1}$. We have

$$
\dfrac{\delta \sigma}{\delta z} = 
    -(1 + \exp(-z))^{-2})(-\exp(-z)) \\
    = \dfrac{\exp(-z)}{(1 + \exp(-z))^2}
$$

For $f(z_1)$, this would correspond to $z_1 = W^1 x + b^1$. We would substitute in the above for $\dfrac{\delta z_2}{\delta z_1}$.

For $g(z_3)$, this would correspond to $z_3 = W^2(f(W^1x + b^1)) + b^2$. We would substitute in the above for $\dfrac{\delta \hat{y}}{\delta z_3}$.

### 1.2.b Now you think you can do a better job by using a binary cross-entropy (BCE) loss function $l_{BCE}(\hat{y}, y) = -[y\log(\hat{y}) + (1-y)\log(1 - \hat{y})]$. What do you need to change in the equations of (b), (c) and (d)?
In the forward pass, we would only have to change the final loss function. 

In the backward pass, we would have to change the $\dfrac{\delta l}{\delta \hat{y}}$ multiple in each of the gradient calculations. The gradient of the cross-entropy loss function with respect to $\hat{y}$ is given by $-(\dfrac{y}{\hat{y}} - \dfrac{1-y}{1 - \hat{y}})$

## 1.2.c
Setting $f$ to be the ReLU function is better for training deeper networks because the ReLU function does not have gradient saturation in the same way the sigmoid function does. The ReLU function's derivative, as claimed earlier, is an indicator function, while the sigmoid function's derivative goes to 0 as the sigmoid's input takes extreme value.

In [63]:
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
from mlp import MLP , mse_loss, bce_loss

import itertools as it
import pandas as pd

In [42]:
def relu(x):
    return x.clamp(min=0)

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def identity(x):
    return x

def d_identity(x):
    return torch.ones_like(x.shape)

def d_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

def d_relu(x):
    return torch.gt(x, 0).int()

class MLP:
    def __init__(
        self,
        linear_1_in_features,
        linear_1_out_features,
        f_function,
        linear_2_in_features,
        linear_2_out_features,
        g_function
    ):
        """
        Args:
            linear_1_in_features: the in features of first linear layer
            linear_1_out_features: the out features of first linear layer
            linear_2_in_features: the in features of second linear layer
            linear_2_out_features: the out features of second linear layer
            f_function: string for the f function: relu | sigmoid | identity
            g_function: string for the g function: relu | sigmoid | identity
        """
        self.f_function = f_function
        self.g_function = g_function

        self.parameters = dict(
            W1 = torch.randn(linear_1_out_features, linear_1_in_features),
            b1 = torch.randn(linear_1_out_features),
            W2 = torch.randn(linear_2_out_features, linear_2_in_features),
            b2 = torch.randn(linear_2_out_features),
        )
        self.grads = dict(
            dJdW1 = torch.zeros(linear_1_out_features, linear_1_in_features),
            dJdb1 = torch.zeros(linear_1_out_features),
            dJdW2 = torch.zeros(linear_2_out_features, linear_2_in_features),
            dJdb2 = torch.zeros(linear_2_out_features),
        )

        # put all the cache value you need in self.cache
        self.cache = dict()
        
        self.function_map = {'relu':relu,
                             'sigmoid':sigmoid,
                             'identity':identity}
        
        self.gradient_map = {'relu':d_relu,
                             'sigmoid':d_sigmoid,
                             'identity':d_identity}
        
        self.f = self.function_map[self.f_function]
        self.g = self.function_map[self.g_function]
        
        self.d_f = self.gradient_map[self.f_function]
        self.d_g = self.gradient_map[self.g_function]

    def forward(self, x):
        """
        Args:
            x: tensor shape (batch_size, linear_1_in_features)
        """
        self.cache['x'] = x
        self.cache['s_1'] = (self.parameters['W1'] @ x.T).T + self.parameters['b1']
        self.cache['z_1'] = self.f(self.cache['s_1'])
        self.cache['s_2'] = (self.parameters['W2'] @ self.cache['z_1'].T).T + self.parameters['b2']
        self.cache['z_2'] = self.g(self.cache['s_2'])
        return self.cache['z_2']
        
    def backward(self, dJdy_hat):
        """
        Args:
            dJdy_hat: The gradient tensor of shape (batch_size, linear_2_out_features)
        """
        self.cache['dz2ds2'] = self.d_g(self.cache['z_2']) # (batch_size, linear_2_out_features)
        self.cache['ds2dW2'] = self.cache['z_1'] # (batch_size, linear_2_out_features, )
        self.cache['ds2db2'] = torch.tensor([1.])
        self.cache['ds2dz1'] = self.parameters['W2']
        self.cache['dz1ds1'] = self.d_f(self.cache['z_1'])
        self.cache['ds1db1'] = torch.tensor([1.])
        self.cache['ds1dw1'] = self.cache['x']
        
        self.grads['dJdW2'] = dJdy_hat @ 
    
    def clear_grad_and_cache(self):
        for grad in self.grads:
            self.grads[grad].zero_()
        self.cache = dict()

In [ ]:
def mse_loss(y, y_hat):
    """
    Args:
        y: the label tensor (batch_size, linear_2_out_features)
        y_hat: the prediction tensor (batch_size, linear_2_out_features)

    Return:
        J: scalar of loss
        dJdy_hat: The gradient tensor of shape (batch_size, linear_2_out_features)
    """
    loss = (0.5 * (y - y_hat) ** 2).mean()
    dJdy_hat = y_hat - y
    return loss, dJdy_hat

In [ ]:
def bce_loss(y, y_hat):
    """
    Args:
        y_hat: the prediction tensor
        y: the label tensor
        
    Return:
        loss: scalar of loss
        dJdy_hat: The gradient tensor of shape (batch_size, linear_2_out_features)
    """
    # TODO: Implement the bce loss
    pass

    # return loss, dJdy_hat

In [61]:
torch.gt(torch.tensor([-1, -2, 0, 1, 2]), 0).int()

tensor([0, 0, 0, 1, 1], dtype=torch.int32)

In [49]:
net = MLP(
    linear_1_in_features=2,
    linear_1_out_features=20,
    f_function='relu',
    linear_2_in_features=20,
    linear_2_out_features=1,
    g_function='identity'
)

In [50]:
x = torch.randn(10, 2)
y = ((torch.randn(10) > 0.5) * 1.0).unsqueeze(-1)

In [51]:
y_hat = net.forward(x)

In [30]:
s = ((net.parameters['W1'] @ x.T).T + net.parameters['b1'])
z = net.f(s)

In [64]:
net_autograd = nn.Sequential(
    OrderedDict([
        ('linear1', nn.Linear(2, 20)),
        ('relu', nn.ReLU()),
        ('linear2', nn.Linear(20, 1)),
    ])
)
net_autograd.linear1.weight.data = net.parameters['W1']
net_autograd.linear1.bias.data = net.parameters['b1']
net_autograd.linear2.weight.data = net.parameters['W2']
net_autograd.linear2.bias.data = net.parameters['b2']

y_hat_autograd = net_autograd(x)
print('y_hat', y_hat)
print('y_hat_autograd', y_hat_autograd)

J_autograd = 0.5 * F.mse_loss(y_hat_autograd, y)

net_autograd.zero_grad()
J_autograd.backward()

y_hat tensor([[-0.4694],
        [-0.7096],
        [ 3.9311],
        [-1.6146],
        [ 0.6903],
        [ 1.1177],
        [-2.1035],
        [ 0.2334],
        [ 2.9613],
        [ 3.6399]])
y_hat_autograd tensor([[-0.4694],
        [-0.7096],
        [ 3.9311],
        [-1.6146],
        [ 0.6903],
        [ 1.1177],
        [-2.1035],
        [ 0.2334],
        [ 2.9613],
        [ 3.6399]], grad_fn=<AddmmBackward>)


In [65]:
print(net_autograd.linear1.weight.grad.data)

tensor([[ 0.0000,  0.0000],
        [-0.2998,  0.8172],
        [ 0.0000,  0.0000],
        [ 0.6248, -1.0746],
        [ 0.0469, -0.1836],
        [ 0.3629, -1.4322],
        [ 0.1679, -0.5939],
        [ 0.3436, -1.6246],
        [-0.2391,  1.1305],
        [ 0.0994, -0.2710],
        [ 0.0448,  0.2470],
        [ 0.1991, -0.9414],
        [ 0.1696, -0.2917],
        [-0.1361,  0.8414],
        [ 0.0224, -0.0384],
        [ 0.1617, -0.3659],
        [ 0.4470, -2.4222],
        [ 0.5801, -2.7433],
        [ 0.0000,  0.0000],
        [-0.0623,  0.2457]])


In [66]:
net_autograd.linear1.weight.grad.data.shape

torch.Size([20, 2])